### Dependencies

In [64]:
import sys
!{sys.executable} -m pip install xgboost
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


### Process Train Set

In [65]:
train_set = pd.read_csv("data/train.csv")
male_sex_df = pd.get_dummies(train_set["Sex"])["male"]
train_set.drop(columns=["PassengerId", "Name", "Cabin", "Ticket", "Sex", "Embarked"], inplace=True)
train_set = pd.concat([train_set, male_sex_df], axis=1)
train_set.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,male
0,0,3,22.0,1,0,7.2500,1
1,1,1,38.0,1,0,71.2833,0
2,1,3,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,0,3,35.0,0,0,8.0500,1


### Train Model

In [66]:
x, y = train_set.drop(columns=["Survived"]), train_set["Survived"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train.head()

,Pclass,Age,SibSp,Parch,Fare,male
498,1,25.0,1,2,151.5500,0
256,1,NaN,0,0,79.2000,0
291,1,19.0,1,0,91.0792,0
130,3,33.0,0,0,7.8958,1
182,3,9.0,4,2,31.3875,1


In [69]:
train = xgb.DMatrix(x_train, label=y_train)

param = {
    "max_depth": 6,
    "eta": 0.05,
    "objective": "multi:softprob",
    "num_class": 3
}
epochs = 300

model = xgb.train(param, train, epochs)
model.save_model("model.json")
print("Model created and saved!!!")

Model created and saved!!!


### Process Test Set

In [115]:
prediction_set = pd.read_csv("data/test.csv")
male_sex_df = pd.get_dummies(prediction_set["Sex"])["male"]
predictions_series = prediction_set["PassengerId"]
prediction_set.drop(columns=["PassengerId", "Name", "Cabin", "Ticket", "Sex", "Embarked"], inplace=True)
prediction_set = pd.concat([prediction_set, male_sex_df], axis=1)
prediction_set.head()

,Pclass,Age,SibSp,Parch,Fare,male
0,3,34.5,0,0,7.8292,1
1,3,47.0,1,0,7.0000,0
2,2,62.0,0,0,9.6875,1
3,3,27.0,0,0,8.6625,1
4,3,22.0,1,1,12.2875,0


#### Run Model

In [121]:
model = xgb.Booster()
model.load_model("model.json")
survived = model.predict(xgb.DMatrix(prediction_set))
prediction_list = np.argmax(survived, axis=1).tolist()
predictions = pd.DataFrame(predictions_series)
predictions["Survived"] = prediction_list
predictions.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,0
4,896,1


In [123]:
predictions.to_csv("predictions.csv", index=False)